# Init
Import all library here!

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor
import numpy as np
from tqdm import tqdm
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import datetime

# Import cleaned data

In [ ]:
camera_df = pd.read_csv("../data/CAMERA/cameras_20221221_202514.csv")
sensor_df = pd.read_csv("../data/SENSOR/sensors_20221221_202410.csv")

In [ ]:
sensor_df['Datetime'] = pd.to_datetime(sensor_df['Datetime'], errors='coerce', format="%Y-%m-%d %H:%M:%S")
camera_df['Datetime'] = pd.to_datetime(camera_df['Datetime'], errors='coerce', format="%Y-%m-%d %H:%M:%S")

In [ ]:
sensor_df.info()

# Group the rows based on CameraID and SensorID

In [ ]:
sensor_gps_groupby_df = sensor_df.groupby(by='SensorCode').agg( {'Latitude': 'mean', 'Longtitude': 'mean' } )
sensor_gps_groupby_df['SensorCode'] = sensor_gps_groupby_df.index
sensor_gps_groupby_df.rename( columns={ 'Latitude': 'LatitudeSensor', 'Longtitude': 'LongtitudeSensor' }, inplace=True )
sensor_gps_groupby_df

In [ ]:
camera_gps_groupby_df = camera_df.groupby(by='CameraCode').agg( {'Latitude': 'mean', 'Longtitude': 'mean' } )
camera_gps_groupby_df['CameraCode'] = camera_gps_groupby_df.index
camera_gps_groupby_df.rename( columns={ 'Latitude': 'LatitudeCam', 'Longtitude': 'LongtitudeCam' }, inplace=True )
camera_gps_groupby_df

In [ ]:
sencam_gps_df = pd.merge(  sensor_gps_groupby_df.assign(joinkey=1),
                                camera_gps_groupby_df.assign(joinkey=1),  
                                on='joinkey').drop('joinkey', axis=1)
sencam_gps_df

In [ ]:
from geopy import distance

In [ ]:
def calculate_geo_distance(x):
    latc, longc, lats, longs = x['LatitudeCam'], x['LongtitudeCam'], x['LatitudeSensor'], x['LongtitudeSensor']
    return distance.distance( 
        ( latc, longc ), 
        ( lats, longs ) ).km

In [ ]:
sencam_gps_df['Distance'] = sencam_gps_df.apply( lambda x: calculate_geo_distance(x), axis=1)

In [ ]:
sencam_gps_df.sort_values(by=['SensorCode','Distance'], ignore_index=True, inplace=True)

In [ ]:
sencam_gps_df.drop(['LatitudeSensor', 'LongtitudeSensor','LatitudeCam', 'LongtitudeCam'], axis=1, inplace=True)

In [ ]:
sencam_gps_df

# Find matches

In [ ]:
matched_sencam_df = sencam_gps_df.copy(deep=True)
matched_sencam_df = matched_sencam_df.drop_duplicates(subset=['SensorCode'], keep='first').reset_index(drop=True)
matched_sencam_df

In [ ]:
for camera in matched_sencam_df.CameraCode.unique():
    cam_used_rows = matched_sencam_df[ matched_sencam_df.CameraCode == camera ]
    print(cam_used_rows, '\n Number of sensor linked: ', len(cam_used_rows), '\n')

### Find the n-th matches

The variable 'nearest_camera_count' are used to decide how many cameras are merged for each sensor. As of this version, we use all of them.

In [ ]:
from functools import reduce

In [ ]:
nearest_camera_count = camera_gps_groupby_df.shape[0]                   # total 15 camera available?
#nearest_camera_count = 5
matches_dfs = [ matched_sencam_df ]
for i in range(2, nearest_camera_count+1):
    # inspired by: https://stackoverflow.com/questions/44706485/how-to-remove-rows-in-a-pandas-dataframe-if-the-same-row-exists-in-another-dataf
    ith_df = pd.merge(sencam_gps_df, pd.concat(matches_dfs), indicator=True, how='left')        # create column '_merge'
    ith_df = ith_df[ ith_df['_merge'] == "left_only" ].drop('_merge', axis=1)       # only the rows from the left are kept, which means discard rows came from the right, the rows of 1st -> (i-1)th distances.
    ith_df.drop_duplicates(subset=['SensorCode'], keep='first', inplace=True)
    #ith_df.reset_index(drop=True, inplace=True)
    matches_dfs.append( ith_df )
    print('\n', str(i), 'th closest \n', ith_df)

In [ ]:
k_matches_sencam_df = pd.concat(matches_dfs).reset_index(drop=True)
k_matches_sencam_df

# save for reporting
os.makedirs("../data/MERGED", exist_ok = True)
k_matches_sencam_df.to_csv('../data/MERGED/sensor_camera_matches.csv', index = False)

# Merge the full data
Note: Following the requirement of this task, I only merge the closest camera into each sensor, not k-nearest cameras. 

In [ ]:
import datetime
min_merge_date = max( datetime.datetime(2022, 8, 13), max( sensor_df.Datetime.sort_values()[0], camera_df.Datetime.sort_values()[0] ) ) 
print(min_merge_date)

In [ ]:
#matched_sencam_df.drop(['Distance'], inplace=True, axis=1)
k_matches_sencam_df.drop(['Distance'], inplace=True, axis=1)

In [ ]:
# firstly merge the sensor name into the camera?
#edited_camera_df = pd.merge(camera_df, k_matches_sencam_df, how='inner', on='CameraCode')
edited_camera_df = camera_df[ camera_df['Datetime'] >= min_merge_date ].copy(True)
edited_camera_df['DatetimeCam'] = edited_camera_df['Datetime']
edited_camera_df.sort_values(by='Datetime', inplace=True, ignore_index=True)
edited_camera_df.rename( columns= {"Latitude": "LatitudeCam", "Longtitude": "LongtitudeCam"} , inplace=True)
edited_camera_df

In [ ]:
edited_sensor_df = sensor_df[ sensor_df['Datetime'] >= min_merge_date ].copy(True)
edited_sensor_df = pd.merge(edited_sensor_df, k_matches_sencam_df, how='inner', on='SensorCode')
edited_sensor_df.sort_values(by='Datetime', ignore_index=True, inplace=True)
edited_sensor_df['DatetimeSen'] = edited_sensor_df['Datetime']
edited_sensor_df.rename( columns={"Latitude": "LatitudeSensor", "Longtitude": "LongtitudeSensor"} , inplace=True)
# edited_sensor_df.head()

### Final touch

In [ ]:
min_driving_speed = 32              #30km/h -> m/s
tolerance_deltatime = datetime.timedelta(0, 5*60, 0)           # now only use at most 5 min delay. Don't know if this is good enough, but let's try it.
#tolerance_deltatime = datetime.timedelta(0, (ext_matched_sencam_df.Distance_1st.max() / min_driving_speed)*3600, 0)
print('Time estimated for vehicles to reach from the sensor to camera position:' , tolerance_deltatime)

In [ ]:
#camera based
sensor_camara_df = pd.merge_asof(edited_camera_df, edited_sensor_df,
                                on='Datetime', by='CameraCode', 
                                direction='nearest', tolerance=tolerance_deltatime)

In [ ]:
# sensor_camara_df = pd.merge_asof(edited_sensor_df, edited_camera_df, on='Datetime', by='SensorCode', direction='nearest', tolerance=tolerance_deltatime)
# sensor_camara_df

In [ ]:
merge_result_df = sensor_camara_df[sensor_camara_df.CameraCode.notna()]
merge_result_df['TimeDistance'] = abs(merge_result_df['Datetime'] - merge_result_df['DatetimeSen'])
print(merge_result_df['TimeDistance'].max(), ' ', merge_result_df['TimeDistance'].min())

In [ ]:
sensor_camara_df.drop( sensor_camara_df.CameraCode[sensor_camara_df.CameraCode.isna()].index, inplace=True)
sensor_camara_df.drop( sensor_camara_df.SensorCode[sensor_camara_df.SensorCode.isna()].index, inplace=True)

In [ ]:
sensor_camara_df.describe()

# Export clean data to CSV
To send to the team.

In [ ]:
output_path = r'../data/MERGED/sensor_camera_' + datetime.datetime.now().strftime(format="%Y%m%d_%H%M")
os.makedirs(output_path, exist_ok=True)

import json
sensor_camara_df.to_csv(os.path.join(output_path, "mergedtable.csv"), index=False)

In [ ]:
with open(os.path.join(output_path, "setting.json"), 'w') as f:
    f.write( json.dumps( { 'nearest_camera_count': nearest_camera_count, 'tolerance_seconds': tolerance_deltatime.total_seconds() }, indent=4 ) )